<a href="https://colab.research.google.com/github/rnop/WSB-Top-Tickers-and-Sentiment-Analysis-Project/blob/main/sentiment_models_BERT_VADER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Compute Sentiment of Comments
* VADER Sentiment
* BERT Transformer

In [ ]:
!pip install transformers
!pip install emoji

     |████████████████████████████████| 3.3 MB 12.4 MB/s 
     |████████████████████████████████| 3.3 MB 78.1 MB/s 
     |████████████████████████████████| 596 kB 53.0 MB/s 
     |████████████████████████████████| 895 kB 55.8 MB/s 
     |████████████████████████████████| 61 kB 473 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 170 kB 11.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=767f40f9f0be51f6c2eacdf5b083e85397d4a73a7c7b9022390471ec2eb4ef90
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
import pandas as pd
import ast

In [ ]:
comments_df = pd.read_csv('submission_comments_tickers.csv')
comments_df['date'] = pd.to_datetime(comments_df['date'])
comments_df.dropna(inplace=True)
comments_df['ticker_in_comment'] = [ast.literal_eval(item) for item in comments_df['ticker_in_comment']] # convert strings to list
comments_df = comments_df[comments_df['comment'].map(len) <= 128] # Remove comments over 128 words for transformer
print("Shape: ", comments_df.shape)
comments_df.head()

Shape:  (199075, 4)


,comment,date,title,ticker_in_comment
0,What am I even supposed to do with myself for ...,2021-01-01,"Daily Discussion Thread for January 01, 2021",[]
1,Goddamn if there’s one sub I can go on to find...,2021-01-01,"Daily Discussion Thread for January 01, 2021",[]
3,“Sorry honey can’t stay up for midnight marke...,2021-01-01,"Daily Discussion Thread for January 01, 2021",[]
4,What if 2020 is so inescapable that today is a...,2021-01-01,"Daily Discussion Thread for January 01, 2021",[]
5,Ended 2020 with 20% loss I am a retard who ma...,2021-01-01,"Daily Discussion Thread for January 01, 2021",[]


In [ ]:
comments_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199075 entries, 0 to 392181
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   comment            199075 non-null  object        
 1   date               199075 non-null  datetime64[ns]
 2   title              199075 non-null  object        
 3   ticker_in_comment  199075 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 7.6+ MB


In [ ]:
from transformers import pipeline

# https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis

model_path = "finiteautomata/bertweet-base-sentiment-analysis"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
sentiment_task("Bullish on SPY 🚀")

[{'label': 'POS', 'score': 0.7241750955581665}]

In [ ]:
# Test run transformer on a sample of the data
test_df = comments_df.sample(3).copy()

bertweet_sentiment_label = []
bertweet_sentiment_score = []
for comment in test_df['comment']:
  sentiment = sentiment_task(comment)

  bertweet_sentiment_label.append(sentiment[0]['label'])
  bertweet_sentiment_score.append(sentiment[0]['score'])

test_df['bertweet_sentiment_label'] = bertweet_sentiment_label
test_df['bertweet_sentiment_score'] = bertweet_sentiment_score

test_df

,comment,date,title,ticker_in_comment,bertweet_sentiment_label,bertweet_sentiment_score
40159,Amc,2021-02-26,"Daily Discussion Thread for February 26, 2021",[AMC],NEU,0.953533
306254,[deleted],2021-08-09,"Daily Discussion Thread for August 09, 2021",[],NEU,0.975669
1876,in week 1,2021-01-04,"Daily Discussion Thread for January 04, 2021",[],NEU,0.973725


In [ ]:
# This cell will take a long time to run ~10hrs on Colab Pro
bertweet_sentiment_label = []
bertweet_sentiment_score = []
for comment in comments_df['comment']:
  sentiment = sentiment_task(comment)

  bertweet_sentiment_label.append(sentiment[0]['label'])
  bertweet_sentiment_score.append(sentiment[0]['score'])

comments_df['bertweet_sentiment_label'] = bertweet_sentiment_label
comments_df['bertweet_sentiment_score'] = bertweet_sentiment_score

In [ ]:
comments_df.sample(10)

,comment,date,title,ticker_in_comment,bertweet_sentiment_label,bertweet_sentiment_score
80913,Market green = GME/AMC red\n\nMarket red = GME...,2021-05-14,"Daily Discussion Thread for May 14, 2021",[],NEG,0.752848
384281,He ruined his life and didn t even have the co...,2021-11-29,"Daily Discussion Thread for November 29, 2021",[],NEG,0.982982
291437,Drill team 6 let’s go,2021-07-16,"Daily Discussion Thread for July 16, 2021",[],NEU,0.877605
323654,Hit ATH buy puts \n\n\nOnce it bottoms out b...,2021-09-01,"Daily Discussion Thread for September 01, 2021",[],NEU,0.981821
372,It smell like Bodussy up in here Booty Dick ...,2021-01-01,"Daily Discussion Thread for January 01, 2021",[],NEG,0.883965
286332,Cool so yesterday was literally just to make m...,2021-07-09,"Daily Discussion Thread for July 09, 2021",[],NEG,0.979444
304942,Should I hold the bag on CRSR ?,2021-08-05,"Daily Discussion Thread for August 05, 2021",[CRSR],NEU,0.981355
82699,Well well well as expected we’re fading into ...,2021-05-18,"Daily Discussion Thread for May 18, 2021",[],NEG,0.560814
293718,Buy spy now or wait till $ 350 pm ?,2021-07-21,"Daily Discussion Thread for July 21, 2021",[SPY],NEU,0.967436
366379,But JPow… I am seeing inflation… In my portf...,2021-11-03,"Daily Discussion Thread for November 03, 2021",[],NEU,0.956525


In [ ]:
comments_df.to_csv('bert_sentiments.csv')

### VADER Sentiment

In [ ]:
! pip install vaderSentiment

     |████████████████████████████████| 125 kB 12.6 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

new_words = {'moon': 4.0,'mooning': 4.0, 'long': 3.0, 'short': -3.0,
             'call': 4.0, 'calls': 4.0, 'put': -4.0, 'puts': -4.0, 
             'break': 2.0, 'tendie': 2.0, 'tendies': 1.0, 'overvalued': -3.0, 
             'undervalued': 3.0, 'buy': 4.0, 'sell': -4.0, 'gone': -2.0, 'gtfo': -1.7, 
             'bullish': 3.7, 'bearish': -3.7, 'bull': 3.7, 'bear': -3.7,
             'bagholder': -3.0, 'stonk': 2.4, 'green': 2.4, 'money': 1.2, 
             'rocket': 3.0, 'pumping': 2.0, 'pump':2.0, 'pamp':2.0,
             'sus': -3.0, 'rip': -4.0, 
             'downgrade': -3.0, 'upgrade': 3.0, 'pump': 2.5, 'hot': 1.5,
             'drop': -2.5, 'rebound': 1.5, 'uppies': 3.0, 'downies': -3.0, '🤡':-3.0, 'clown':-3.0,
             '🌈🐻': -3.0, 'rainbow bear':-3.0, 'fire':-2.0,
             }

analyser.lexicon.update(new_words)

comments_df['vader_sentiment'] = comments_df.apply(lambda x: analyser.polarity_scores(x['comment'])['compound'], axis=1)
comments_df.to_csv('sentiment_comments.csv', index=False)

In [ ]:
#Vader test
analyser.polarity_scores("Bearish SPY but neutral")

{'compound': -0.431, 'neg': 0.487, 'neu': 0.513, 'pos': 0.0}

In [ ]:
vader_label = []
for vader_score in comments_df['vader_sentiment']:
  if vader_score > 0.15:
    vader_label.append('POS')
  elif vader_score < -0.15:
    vader_label.append('NEG')
  else:
    vader_label.append('NEU')

comments_df['vader_sentiment_label'] = vader_label

In [ ]:
comments_df.sample(10)

,comment,date,title,ticker_in_comment,bertweet_sentiment_label,bertweet_sentiment_score,vader_sentiment,vader_sentiment_label
131435,You’re welcome to everyone holding Tesla cause...,2021-05-24,"Daily Discussion Thread for May 24, 2021",[],POS,0.986791,0.6124,POS
17519,Just bought my first stock ever Bb and NOK le...,2021-01-27,"Daily Discussion Thread for January 27, 2021 -...","[NOK, BB]",POS,0.673409,0.0000,NEU
313752,[deleted],2021-08-18,"Daily Discussion Thread for August 18, 2021",[],NEU,0.975669,0.0000,NEU
294076,SOFI IS SOFLY,2021-07-21,"Daily Discussion Thread for July 21, 2021",[SOFI],NEU,0.524856,0.0000,NEU
13996,Bought 2k worth of BB shares Let’s fucking go...,2021-01-21,"Daily Discussion Thread for January 21, 2021",[BB],POS,0.686041,0.4767,POS
45196,RBLX 🚀,2021-03-10,"Daily Discussion Thread for March 10, 2021",[],NEU,0.977487,0.6124,POS
68730,I don t know why BB is shooting all of a sudde...,2021-04-27,"Daily Discussion Thread for April 27, 2021",[BB],POS,0.965623,0.5023,POS
323095,[deleted],2021-09-01,"Daily Discussion Thread for September 01, 2021",[],NEU,0.975669,0.0000,NEU
271277,I ll Burry my nuts in the bears faces,2021-06-17,"Daily Discussion Thread for June 17, 2021",[],NEG,0.804655,-0.3182,NEG
297863,This hurts much more than anticipated,2021-07-27,"Daily Discussion Thread for July 27, 2021",[],NEG,0.975427,-0.4767,NEG


In [ ]:
comments_df.to_csv('bert_vader_sentiment_comments.csv', index=False)